In [1]:
import os

import torch
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer

In [2]:
# prevent env load failed
%load_ext dotenv
%dotenv

In [3]:
login(token=os.environ.get("HF_TOKEN", ""), add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/hermeschen/.cache/huggingface/token
Login successful


# Load Dataset

In [ ]:
dataset = load_dataset("daily_dialog",
                       split="train+validation",
                       num_proc=16,
                       trust_remote_code=True).remove_columns("act")

In [ ]:
dataset = dataset.rename_column("emotion", "emotion_id")
emotion_labels: list = dataset.features["emotion_id"].feature.names
emotion_labels[0] = "neutral"
dataset = dataset.map(lambda samples: {
    "emotion": [[emotion_labels[emotion_id] for emotion_id in sample] for sample in samples]
}, input_columns="emotion_id", remove_columns="emotion_id", batched=True, num_proc=16)
dataset = dataset.map(lambda samples: {
    "emotion": [sample[:-1] for sample in samples if len(sample) % 2]
}, input_columns="emotion", batched=True, num_proc=16)

In [ ]:
dataset = dataset.map(lambda samples: {
    "dialog": [[dialog.strip() for dialog in sample] for sample in samples]
}, input_columns="dialog", batched=True, num_proc=16)
dataset = dataset.map(lambda samples: {
    "dialog": [sample[:-1] for sample in samples if len(sample) % 2]
}, input_columns="dialog", batched=True, num_proc=16)

In [11]:
dataset = dataset.map(lambda samples: {
    "prompt": [[{"role": "user" if i % 2 == 0 else "assistant", "emotion": emotion, "dialog": dialog}
                for i, emotion, dialog in enumerate(zip(sample[0], sample[1]))]
               for sample in zip(samples["emotion"], samples["dialog"])]
}, remove_columns=["emotion", "dialog"], batched=True, num_proc=16)

Map (num_proc=16):   0%|          | 0/12118 [00:00<?, ? examples/s]

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
base_model_name: str = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True,
    clean_up_tokenization_spaces=True,
    add_special_tokens=True,
    padding_side="right",
    truncation=True,
    truncation_side="right")
tokenizer.eos_token = "<eos>" if tokenizer.eos_token is None else tokenizer.eos_token
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

In [ ]:
chat_template: str = """
"""

In [7]:
tokenizer.apply_chat_template(dataset[0]["prompt"],
                              chat_template=chat_template,
                              add_generation_prompt=True)


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must

## Configurations

In [26]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

In [27]:
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

In [28]:
num_train_epochs: int = 3

In [29]:
train_params = TrainingArguments(
    output_dir=f"./checkpoints_{fine_tuned_model_name}",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to=["tensorboard"],
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": True}
)

## Load Model

In [30]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quantization_config if torch.cuda.is_available() else None,
    device_map="auto" if torch.cuda.is_available() else "cpu",
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

## Setup Tuner

In [32]:
tuner = SFTTrainer(
    model=base_model,
    train_dataset=train_data,
    dataset_text_field="prompt",
    tokenizer=tokenizer,
    peft_config=peft_parameters,
    args=train_params,
    max_seq_length=1024
)

In [33]:
tuner.train()

Step,Training Loss
25,1.211500
50,0.911800
75,0.911400
100,0.843400
125,0.920900
150,0.843200
175,0.909000
200,0.856900
225,0.900200
250,0.831600


/home/hermeschen/.cache/pypoetry/virtualenvs/chat-bot-uhayQKRl-py3.11/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e445f5bf-db2f-4edb-a19e-051f583fee7a)') - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
/home/hermeschen/.cache/pypoetry/virtualenvs/chat-bot-uhayQKRl-py3.11/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=8340, training_loss=0.79868845985376, metrics={'train_runtime': 63106.1914, 'train_samples_per_second': 0.529, 'train_steps_per_second': 0.132, 'total_flos': 3.844355152962355e+17, 'train_loss': 0.79868845985376, 'epoch': 3.0})

In [34]:
tuner.model.save_pretrained(fine_tuned_model_name)